In [ ]:
import json
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich import box
import time
import hashlib
import hmac
import requests
import json

In [ ]:
def gen_sign(method, url, query_string=None, payload_string=None):
    sign_path = "../cred/credentials.json"

    # Read the credentials from the file
    with open(sign_path, "r") as f:
        data = json.load(f)
        key = data["pub_key"]
        secret = data["secret_key"]

    t = time.time()
    m = hashlib.sha512()
    m.update((payload_string or "").encode("utf-8"))
    hashed_payload = m.hexdigest()
    s = "%s\n%s\n%s\n%s\n%s" % (method, url, query_string or "", hashed_payload, t)
    sign = hmac.new(
        secret.encode("utf-8"), s.encode("utf-8"), hashlib.sha512
    ).hexdigest()
    return {"KEY": key, "Timestamp": str(t), "SIGN": sign}


def get_data():
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {"Accept": "application/json", "Content-Type": "application/json"}

    # url = '/account/detail'
    url = "/futures/usdt/contract_stats"
    query_param = "contract=BTC_USDT"
    r = requests.request(
        "GET", host + prefix + url + "?" + query_param, headers=headers
    )
    # print(r.json())
    return r.json()

In [ ]:
data = get_data()
type(data)

In [ ]:
len(data)

In [ ]:
# coding: utf-8
import requests
import time
import hashlib
import hmac

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

url = "/futures/usdt/positions"
query_param = ""
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign("GET", prefix + url, query_param)
headers.update(sign_headers)
r = requests.request("GET", host + prefix + url, headers=headers)
data = r.json()
print(data)


# summarize above as a function
def get_position_now(url="/futures/usdt/positions"):
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {"Accept": "application/json", "Content-Type": "application/json"}

    url = "/futures/usdt/positions"
    query_param = ""
    sign_headers = gen_sign("GET", prefix + url, query_param)
    headers.update(sign_headers)
    r = requests.request("GET", host + prefix + url, headers=headers)
    data = r.json()
    result_data = [
        (v["contract"], v["value"], v["realised_pnl"], v["unrealised_pnl"])
        for v in data
        if v["value"] != "0"
    ]
    return result_data

In [ ]:
# Show all columns
# [v for v in data[0]]
data = get_position_now()
data

In [ ]:
def get_data():
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {"Accept": "application/json", "Content-Type": "application/json"}

    # url = '/account/detail'
    url = "/wallet/total_balance"
    query_param = ""
    sign_headers = gen_sign("GET", prefix + url, query_param)
    headers.update(sign_headers)
    r = requests.request("GET", host + prefix + url, headers=headers)
    # print(r.json())
    return r.json()


def fetch_account_info_test():
    res = get_data()
    return {
        "details": {
            "delivery": {
                "currency": "USDT",
                "amount": res["details"]["delivery"]["amount"],
                "unrealised_pnl": res["details"]["delivery"]["unrealised_pnl"],
            },
            "finance": {
                "currency": "USDT",
                "amount": res["details"]["finance"]["amount"],
            },
            "futures": {
                "currency": "USDT",
                "amount": res["details"]["futures"]["amount"],
                "unrealised_pnl": res["details"]["futures"]["unrealised_pnl"],
            },
            "margin": {
                "currency": "USDT",
                "amount": res["details"]["margin"]["amount"],
                "borrowed": res["details"]["margin"]["borrowed"],
            },
            "options": {
                "currency": "USDT",
                "amount": res["details"]["options"]["amount"],
                "unrealised_pnl": res["details"]["options"]["unrealised_pnl"],
            },
            "payment": {
                "currency": "USDT",
                "amount": res["details"]["payment"]["amount"],
            },
            "quant": {"currency": "USDT", "amount": res["details"]["quant"]["amount"]},
            "spot": {"currency": "USDT", "amount": res["details"]["spot"]["amount"]},
        },
        "total": {
            "amount": res["total"]["amount"],
            "borrowed": res["total"]["borrowed"],
            "currency": res["total"]["currency"],
            "unrealised_pnl": res["total"]["unrealised_pnl"],
        },
    }


# Mock-up function for fetching data - replace this with your actual fetching logic
def fetch_account_info():
    res = get_data()
    return {
        "details": {
            "delivery": {
                "currency": "USDT",
                "amount": res["details"]["delivery"]["amount"],
                "unrealised_pnl": res["details"]["delivery"]["unrealised_pnl"],
            },
            "finance": {
                "currency": "USDT",
                "amount": res["details"]["finance"]["amount"],
            },
            "futures": {
                "currency": "USDT",
                "amount": res["details"]["futures"]["amount"],
                "unrealised_pnl": res["details"]["futures"]["unrealised_pnl"],
            },
            "margin": {
                "currency": "USDT",
                "amount": res["details"]["margin"]["amount"],
                "borrowed": res["details"]["margin"]["borrowed"],
            },
            "options": {
                "currency": "USDT",
                "amount": res["details"]["options"]["amount"],
                "unrealised_pnl": res["details"]["options"]["unrealised_pnl"],
            },
            "payment": {
                "currency": "USDT",
                "amount": res["details"]["payment"]["amount"],
            },
            "quant": {"currency": "USDT", "amount": res["details"]["quant"]["amount"]},
            "spot": {"currency": "USDT", "amount": res["details"]["spot"]["amount"]},
        },
        "total": {
            "amount": res["total"]["amount"],
            "borrowed": res["total"]["borrowed"],
            "currency": res["total"]["currency"],
            "unrealised_pnl": res["total"]["unrealised_pnl"],
        },
    }


def get_position_now(url="/futures/usdt/positions"):
    try:
        host = "https://api.gateio.ws"
        prefix = "/api/v4"
        headers = {"Accept": "application/json", "Content-Type": "application/json"}

        sign_headers = gen_sign("GET", prefix + url, "")
        headers.update(sign_headers)
        response = requests.get(host + prefix + url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        # Filter and return positions with non-zero value
        return [
            (v["contract"], v["value"], v["realised_pnl"], v["unrealised_pnl"])
            for v in data
            if v["value"] != "0"
        ]
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return []


def display_account_info(account_info):
    console = Console()

    # Display account details using Rich
    details_table = Table(
        show_header=True, header_style="bold magenta", box=box.DOUBLE_EDGE
    )
    details_table.add_column("Section 📂", style="dim", width=12)
    details_table.add_column("Currency 💵", min_width=10)
    details_table.add_column("Amount 💰", min_width=15)
    details_table.add_column("Unrealised PnL 📉", min_width=20)
    for section, info in account_info["details"].items():
        emoji = "🔵" if float(info["amount"]) > 0 else "🔴"
        pnl_emoji = "🟢" if float(info.get("unrealised_pnl", "0")) >= 0 else "🔴"
        details_table.add_row(
            f"{emoji} {section.capitalize()}",
            info["currency"],
            f"{emoji} {info['amount']}",
            f"{pnl_emoji} {info.get('unrealised_pnl', 'N/A')}",
        )
    console.print(
        Panel(
            details_table,
            title=f"[bold cyan]Account Details[/]",
            subtitle="Sections Overview",
            expand=False,
        )
    )

    # Fetch and display current positions
    positions = get_position_now()
    if positions:
        positions_table = Table(
            show_header=True, header_style="bold blue", box=box.SQUARE
        )
        positions_table.add_column("Contract 📝", style="dim", width=20)
        positions_table.add_column("Value 💵", width=15)
        positions_table.add_column("Realised PnL 📘", width=15)
        positions_table.add_column("Unrealised PnL 📗", width=15)
        for contract, value, realised_pnl, unrealised_pnl in positions:
            value_emoji = "🔵" if float(value) > 0 else "🔴"
            realised_pnl_emoji = "🟢" if float(realised_pnl) >= 0 else "🔴"
            unrealised_pnl_emoji = "🟢" if float(unrealised_pnl) >= 0 else "🔴"
            positions_table.add_row(
                f"{contract}",
                f"{value_emoji} {value}",
                f"{realised_pnl_emoji} {realised_pnl}",
                f"{unrealised_pnl_emoji} {unrealised_pnl}",
            )
        console.print(
            Panel(
                positions_table,
                title="[bold green]Current Positions[/]",
                subtitle="Futures Contracts Overview",
                expand=False,
            )
        )


# get_position_now()

display_account_info(fetch_account_info())

In [ ]:
# Show position which value is not 0 in contract real time
[
    (v["contract"], v["value"], v["realised_pnl"], v["unrealised_pnl"])
    for v in data
    if v["value"] != "0"
]

In [ ]:
[v for v in data if v["value"] != "0"]

In [ ]:
# Show contract history:
# # coding: utf-8
# import requests
# import time
# import hashlib
# import hmac
import json


host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

url = "/futures/usdt/my_trades"
query_param = ""
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign("GET", prefix + url, query_param)
headers.update(sign_headers)
r = requests.request("GET", host + prefix + url, headers=headers)
print(r.json())


# package above as a function
def get_history_contract(url="/futures/usdt/my_trades"):
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {"Accept": "application/json", "Content-Type": "application/json"}
    query_param = ""
    sign_headers = gen_sign("GET", prefix + url, query_param)
    headers.update(sign_headers)
    r = requests.request("GET", host + prefix + url, headers=headers)
    return r.json()

In [ ]:
history_data = get_history_contract()
len(history_data)
df = pd.DataFrame(history_data)

In [ ]:
history_data[0]

In [ ]:
# 查询个人成交记录(时间区间)
# coding: utf-8
import requests
import time
import hashlib
import hmac

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

url = "/futures/usdt/my_trades_timerange"
query_param = ""
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign("GET", prefix + url, query_param)
headers.update(sign_headers)
r = requests.request("GET", host + prefix + url, headers=headers)
print(r.json())

In [ ]:
# 平倉歷史
host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

url = "/futures/usdt/position_close"
query_param = ""
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign("GET", prefix + url, query_param)
headers.update(sign_headers)
r = requests.request("GET", host + prefix + url, headers=headers)
print(r.json())
print(len(r.json()))

In [ ]:
# Getting the commission history
# coding: utf-8
import requests
import time
import hashlib
import hmac

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

url = "/rebate/agency/commission_history"
query_param = ""
# `gen_sign` 的实现参考认证一章
sign_headers = gen_sign("GET", prefix + url, query_param)
headers.update(sign_headers)
r = requests.request("GET", host + prefix + url, headers=headers)
print(r.json())

In [ ]:
# coding: utf-8
import requests

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

url = "/futures/usdt/contracts"
query_param = ""
r = requests.request("GET", host + prefix + url, headers=headers)
data = r.json()
# print(data)
# print(len(r.json()))
# print(len(data))

from sqlalchemy import create_engine
import pandas as pd
df = pd.DataFrame(data)
# df.columns

In [ ]:
[v for v in df.columns]

In [ ]:
pd.to_datetime(df['config_change_time'], unit='s')

In [ ]:
target_list = ['ETH_USDT', 'BTC_USDT', 'DOGE_USDT']
result = [v for v in data if v['name'] in target_list]  
len(result)
result

In [ ]:
import pandas as pd


df = pd.DataFrame(result)
df.head()
df.columns

In [ ]:
df[['name', 'last_price','mark_price', 'index_price', 'interest_rate', 'trade_id']]

In [ ]:
[v["name"] for v in data]

In [ ]:
# coding: utf-8
import requests

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

url = "/futures/usdt/tickers"
query_param = ""
r = requests.request("GET", host + prefix + url, headers=headers)
data = r.json()
print(data)
print(len(data))

In [ ]:
[v for v in data if v["contract"] == "BTC_USDT"]

In [ ]:
import requests


def get_btc_usdt_data():
    url = "https://api.gateio.ws/api/v4/spot/tickers"
    params = {"currency_pair": "BTC_USDT"}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an exception for 4XX or 5XX errors
        data = response.json()
        return data
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None


# Fetch and display the BTC/USDT data
btc_usdt_data = get_btc_usdt_data()
if btc_usdt_data:
    print(btc_usdt_data)

In [ ]:
btc_usdt_data

In [ ]:
def display_account_info(account_info):
    console = Console()

    # Display account details using Rich
    details_table = Table(
        show_header=True, header_style="bold magenta", box=box.DOUBLE_EDGE
    )
    details_table.add_column("Section 📂", style="dim", width=12)
    details_table.add_column("Currency 💵", min_width=10)
    details_table.add_column("Amount 💰", min_width=15)
    details_table.add_column("Unrealised PnL 📉", min_width=20)
    for section, info in account_info["details"].items():
        emoji = "🔵" if float(info["amount"]) > 0 else "🔴"
        pnl_emoji = "🟢" if float(info.get("unrealised_pnl", "0")) >= 0 else "🔴"
        details_table.add_row(
            f"{emoji} {section.capitalize()}",
            info["currency"],
            f"{emoji} {info['amount']}",
            f"{pnl_emoji} {info.get('unrealised_pnl', 'N/A')}",
        )
    console.print(
        Panel(
            details_table,
            title=f"[bold cyan]Account Details[/]",
            subtitle="Sections Overview",
            expand=False,
        )
    )

    # Fetch and display current positions
    positions = get_position_now()
    if positions:
        positions_table = Table(
            show_header=True, header_style="bold blue", box=box.SQUARE
        )
        positions_table.add_column("Contract 📝", style="dim", width=20)
        positions_table.add_column("Value 💵", width=15)
        positions_table.add_column("Realised PnL 📘", width=15)
        positions_table.add_column("Unrealised PnL 📗", width=15)
        for contract, value, realised_pnl, unrealised_pnl in positions:
            value_emoji = "🔵" if float(value) > 0 else "🔴"
            realised_pnl_emoji = "🟢" if float(realised_pnl) >= 0 else "🔴"
            unrealised_pnl_emoji = "🟢" if float(unrealised_pnl) >= 0 else "🔴"
            positions_table.add_row(
                f"{contract}",
                f"{value_emoji} {value}",
                f"{realised_pnl_emoji} {realised_pnl}",
                f"{unrealised_pnl_emoji} {unrealised_pnl}",
            )
        console.print(
            Panel(
                positions_table,
                title="[bold green]Current Positions[/]",
                subtitle="Futures Contracts Overview",
                expand=False,
            )
        )